In [ ]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import sys
import torch
#import scripts.zero_shot_reconstruction as zs
from torch.utils.data import DataLoader, TensorDataset
from gevar.data import WindTurbineDataset
# from notebooks.helpers import get_all_cross_corr_features, compute_max_cross_corr
# from notebooks.helpers import plot_mse_by_channel_with_total
# from notebooks.helpers import apply_pca_to_frames
from momentfm import MOMENTPipeline
from scipy.stats import multivariate_normal
from scipy.ndimage import gaussian_filter1d
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.patches as patches

In [ ]:
def create_data(n_examples, dimension, control_range, anomaly_value=1, noise_level = .25, control_noise_level = .01, seed=1):
    torch.manual_seed(seed)
    slices = [slice(i*dimension, (i+1)*dimension) for i in range(3)]
    sig_len = dimension * 3
    control_data = torch.zeros(n_examples, sig_len)
    sensor_data = torch.zeros(n_examples, sig_len)
    rand_v = torch.rand((n_examples,3)) * (control_range[1] - control_range[0]) + control_range[0]
    for i in range(3):
        control_data[:, slices[i]] = rand_v[:,i][:, None]

    offsets = torch.rand(n_examples,)*6.28
    for i in range(3):
        sin_in = offsets[:, None] + anomaly_value*rand_v[:, i][:, None]*(torch.arange(dimension+1)[None, :].float() / dimension * 6.28*10)
        sin_in = sin_in[:, 1:]
        sin = torch.sin(sin_in)
        sensor_data[:, slices[i]] = sin

        cos = torch.cos(sin_in)
        offsets = torch.atan2(sin[:,-1], cos[:,-1])
        print(offsets)
        offsets[offsets < 0] += 6.28  # Ensure offsets are in [0, 2*pi]

        #sensor_data[:, slices[i]] = torch.sin(offsets[:, None] + control_data[:,slices[i]])
        #offsets = torch.asin(sensor_data[:, (i+1)*dimension-1])
        #is_down = sensor_data[:, (i+1)*dimension-1] - sensor_data[:, (i+1)*dimension-2] < 0
        #offsets[is_down] = offsets[is_down] + 3.14
       # print(offsets[0:4])

    sensor_data += torch.randn_like(sensor_data) * noise_level
    control_data += torch.randn_like(control_data) * control_noise_level
    return control_data, sensor_data

In [ ]:
control_data_train, sensor_data_train = create_data(10000, 170, (0, 1), anomaly_value=1, seed=1)
control_data_test, sensor_data_test = create_data(10000, 170, (0, 1), anomaly_value=1, seed=2)

In [ ]:
fig, ax = plt.subplots(2, 1)
ax[0].plot(control_data_train[10, :].numpy())
ax[0].set_ylabel('Control Frequency')
ax[1].plot(sensor_data_train[10, :].numpy())
ax[1].set_ylabel('Sensor Frequency')
ax[1].set_xlabel('Time')

In [ ]:
model = MOMENTPipeline.from_pretrained(
        "AutonLab/MOMENT-1-base", #("AutonLab/MOMENT-1-large")
        model_kwargs={"task_name": 'embedding'},
    )
embed_dim=512+256
model.init()
dir(model)

device = torch.device("cuda")
model = model.to(device)

In [ ]:

def compute_embeddings(control, sensor, batch_size=512):
    embeddings = torch.zeros(control.shape[0], embed_dim*2, device=device)
    dataset = TensorDataset(control, sensor)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    with torch.no_grad():
        for i, (control_batch, sensor_batch) in enumerate(tqdm(dataloader)):
            control_batch = control_batch.to(device)
            sensor_batch = sensor_batch.to(device)
            control_embeddings = model(x_enc=control_batch.unsqueeze(1)).embeddings
            sensor_embeddings = model(x_enc=sensor_batch.unsqueeze(1)).embeddings
            embeddings[i * batch_size:(i + 1) * batch_size, 0:embed_dim] = control_embeddings
            embeddings[i * batch_size:(i + 1) * batch_size, embed_dim:] = sensor_embeddings

    return embeddings.cpu().numpy()

In [ ]:
embeddings_train = compute_embeddings(control_data_train, sensor_data_train)
embeddings_test = compute_embeddings(control_data_test, sensor_data_test)

In [ ]:
anomaly_value = .1
control_data_anomaly, sensor_data_anomaly = create_data(10000, 170, (0, 1), anomaly_value=anomaly_value, seed=3)
embeddings_anomaly = compute_embeddings(control_data_anomaly, sensor_data_anomaly)

In [ ]:
data_train = torch.cat((control_data_train, sensor_data_train), dim=1)
data_test = torch.cat((control_data_test, sensor_data_test), dim=1)
data_anomaly = torch.cat((control_data_anomaly, sensor_data_anomaly), dim=1)

In [ ]:
control_data_test.shape

In [ ]:
mvn_data = multivariate_normal(mean=data_train.mean(axis=0), cov=np.cov(data_train.T))
mvn_embed = multivariate_normal(mean=embeddings_train.mean(axis=0), cov=np.cov(embeddings_train.T))

p_data_train = mvn_data.logpdf(data_train.numpy())
p_data_test = mvn_data.logpdf(data_test.numpy())
p_data_anomaly = mvn_data.logpdf(data_anomaly.numpy())

p_embeddings_train = mvn_embed.logpdf(embeddings_train)
p_embeddings_test = mvn_embed.logpdf(embeddings_test)
p_embeddings_anomaly = mvn_embed.logpdf(embeddings_anomaly)

In [ ]:
plt.hist(p_data_train, bins=100, alpha=0.5, label='Train Data')
plt.hist(p_data_test, bins=100, alpha=0.5, label='Test Data')
plt.hist(p_data_anomaly, bins=100, alpha=0.5, label='Anomaly Data')
plt.legend()
plt.xlabel('Log Probability')
plt.ylabel('Frequency')
plt.title(f'Log Probability Distribution of Data\n Anomaly Value: {anomaly_value}')
plt.savefig(f'../../plots/p_data_{anomaly_value}.png')
plt.show()

In [ ]:
_, bins, _ = plt.hist(p_embeddings_train, bins=100, alpha=0.5, label='Train Embeddings')
plt.hist(p_embeddings_test, bins=bins, alpha=0.5, label='Test Embeddings')
plt.hist(p_embeddings_anomaly, bins=bins, alpha=0.5, label='Anomaly Embeddings')
plt.legend()
plt.xlabel('Log Probability')
plt.ylabel('Frequency')
plt.title(f'Log Probability Distribution of Embeddings\n Anomaly Value: {anomaly_value}')
plt.savefig(f'../../plots/p_embed_{anomaly_value}.png')
#plt.xscale('log')
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, auc
fpr, tpr, thresholds = roc_curve(np.concatenate([np.zeros(len(p_embeddings_test)),
                                                  np.ones(len(p_embeddings_anomaly))]),
                                 np.concatenate([-p_embeddings_test, -p_embeddings_anomaly]))
fpr_data, tpr_data, thresholds_data = roc_curve(np.concatenate([np.zeros(len(p_data_test)),
                                                  np.ones(len(p_data_anomaly))]),
                                 np.concatenate([-p_data_test, -p_data_anomaly]))
plt.plot(fpr, tpr, label='Embeddings ROC (AUC = {:.2f})'.format(auc(fpr, tpr)))
plt.plot(fpr_data, tpr_data, label='Data ROC (AUC = {:.2f})'.format(auc(fpr_data, tpr_data)))
plt.plot([0, 1], [0, 1], 'k--', label='Random Guessing')
plt.xlabel('False Anomaly Rate')
plt.ylabel('True Anomaly Rate')
plt.legend()
plt.savefig(f'../../plots/roc_{anomaly_value}.png')

plt.title(f'ROC Curve for Anomaly Detection \n Anomaly Value: {anomaly_value}')

In [ ]:
pca_embed = PCA(n_components=embeddings_train.shape[1])
pca_embed.fit(embeddings_train)
pca_coeff_embed_test = pca_embed.transform(embeddings_test)
pca_coeff_embed_anomaly = pca_embed.transform(embeddings_anomaly)

for k in [0, 1, 2, 3, 5, 10, 20, 40, 80, 160, 300, 500]:
    with_zeros_test = pca_coeff_embed_test.copy()
    with_zeros_test[:, k:] = 0
    pca_recon_embed_test = pca_embed.inverse_transform(with_zeros_test)

    with_zeros_anomaly = pca_coeff_embed_anomaly.copy()
    with_zeros_anomaly[:, k:] = 0
    pca_recon_embed_anomaly = pca_embed.inverse_transform(with_zeros_anomaly)

    fpr, tpr, thresholds = roc_curve(np.concatenate([np.zeros(len(pca_recon_embed_test)),
                                                    np.ones(len(pca_recon_embed_anomaly))]),
                                    np.concatenate([np.linalg.norm(pca_recon_embed_test, axis=1),
                                                    np.linalg.norm(pca_recon_embed_anomaly, axis=1)]))

    plt.plot(fpr, tpr, label='PCA Embed k={} ROC (AUC = {:.2f})'.format(k, auc(fpr, tpr)))
    plt.title(f'PCA Embedding ROC Curve \n Anomaly Value: {anomaly_value}')
    plt.savefig(f'../../plots/pca_embed_roc_{anomaly_value}.png')
    plt.legend()